In [9]:
import os
from skimage import io, transform
import numpy as np
from tqdm import tqdm
from model import FireNet
from data_augmentation import prepare_dataset

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from torch.utils.tensorboard import SummaryWriter

In [10]:
prepare_dataset()

DOWNLOADING TRAINING DATASET



KeyboardInterrupt: 

In [3]:
writer = SummaryWriter('runs/firenet_experiment_1')

In [4]:
print(torch.cuda.is_available())
net = FireNet()
device = "cuda" if torch.cuda.is_available() else "cpu"
net.to(device)

True


FireNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (drop1): Dropout2d(p=0.5, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=2304, out_features=256, bias=True)
  (dense2): Linear(in_features=256, out_features=128, bias=True)
  (dense3): Linear(in_features=128, out_features=2, bias=True)
)

In [5]:
TRAINING_PATH='training_dataset'
CATEGORIES = ['Fire', 'NoFire']

In [6]:
class TrainingSet(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        classes = []
        images = []
        for category in CATEGORIES:
            path = os.path.join(TRAINING_PATH, category)
            class_num = CATEGORIES.index(category)
            
            for img in tqdm(os.listdir(path)):
                try:
                    image = io.imread(os.path.join(path, img))
                    if (image.shape[2] == 3):
                        if (self.transform is not None):
                            image = self.transform(image)
                            images.append(image)
                            classes.append(class_num)
                except Exception as e:
                    pass
        self.set = {'image': images, 'class': classes}
    def __len__(self):
        return len(self.set['class'])
    def __getitem__(self, idx):
        image = self.set['image'][idx]
        classe = self.set['class'][idx]
        sample = {'image': image, 'class': classe}
        return sample
            
        

In [7]:
training_set= TrainingSet(transform=transforms.ToTensor())
print(len(training_set))

100%|██████████| 5172/5172 [00:01<00:00, 2655.68it/s]

9616


In [8]:
class TestingSet(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        classes = []
        images = []
        for category in CATEGORIES:
            path = os.path.join('testing_dataset', category)
            class_num = CATEGORIES.index(category)
            
            for img in tqdm(os.listdir(path)):
                try:
                    image = io.imread(os.path.join(path, img))
                    if (image.shape[2] == 3):
                        if (self.transform):
                            image = self.transform(image)
                            images.append(image)
                            classes.append(class_num)
                except Exception as e:
                    pass
        self.set = {'image': images, 'class': classes}
    def __len__(self):
        return len(self.set['class'])
    def __getitem__(self, idx):
        image = self.set['image'][idx]
        classe = self.set['class'][idx]
        sample = {'image': image, 'class': classe}
        return sample
            
        

In [9]:
test_set = TestingSet(transform=transforms.ToTensor())
print(len(test_set))

100%|██████████| 100/100 [00:00<00:00, 2650.91it/s]

200


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, eps=1e-7, weight_decay=1e-7)

In [11]:
trainloader = DataLoader(training_set, batch_size=32, shuffle=True, num_workers=4)
net = net.float()

In [12]:
len(trainloader)

301

In [13]:
for epoch in range(100):
    global_loss = 0.0
    for data in tqdm(trainloader):
        inputs, labels = data['image'].to(device), data['class'].to(device)

        optimizer.zero_grad()

        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        global_loss += loss.item()
    print('global loss for epoch %d : %.3f' % (epoch + 1, global_loss/ len(trainloader)))
    #writer.add_scalar('Training loss 300', global_loss / len(trainloader), epoch + 1)

  0%|          | 0/301 [00:00<?, ?it/s]

global loss for epoch 1 : 0.572


  0%|          | 0/301 [00:00<?, ?it/s]

global loss for epoch 2 : 0.505


  0%|          | 0/301 [00:00<?, ?it/s]

...

global loss for epoch 298 : 0.101


  0%|          | 0/301 [00:00<?, ?it/s]

global loss for epoch 299 : 0.104


100%|██████████| 301/301 [00:01<00:00, 159.34it/s]

global loss for epoch 300 : 0.098


In [14]:
torch.save(net.state_dict(), './trained_weights100.pth')

In [31]:
net = FireNet()
net.float()
net.cuda()
net.load_state_dict(torch.load('./trained_weights100.pth'))

<All keys matched successfully>

In [32]:
trainloader = DataLoader(training_set, batch_size=4, shuffle=False, num_workers=0)
net = net.float()

In [33]:
correct = 0
total = 0
with torch.no_grad():
    print('evaluate accuracy on training set:')
    for data in tqdm(trainloader):
        images, labels = data['image'].to(device), data['class'].to(device)
        outputs = net(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()
print ('accuracy: %d' % (100*correct/total))

  5%|▍         | 115/2404 [00:00<00:01, 1144.55it/s]

evaluate accuracy on training set:


100%|██████████| 2404/2404 [00:01<00:00, 1219.96it/s]

accuracy: 96


In [34]:
testloader = DataLoader(test_set, batch_size=4, shuffle=False, num_workers=0)
net = net.float()

In [35]:
len(testloader)

50

In [43]:
correct = 0
total = 0
with torch.no_grad():
    print('evaluate accuracy on training set:')
    for data in tqdm(testloader):
        images, labels = data['image'].to(device), data['class'].to(device)
        outputs = net(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()
print ('accuracy: %d' % (100*correct/total))

100%|██████████| 50/50 [00:00<00:00, 1093.85it/s]

evaluate accuracy on training set:
accuracy: 76
